In [ ]:
import numpy as np
import pandas as pd
import io
import requests

# Registry of supported datasets
DATASETS = {
    "residential": "https://archive.ics.uci.edu/ml/machine-learning-databases/00437/Residential-Building-Data-Set.xlsx",
    "secom": "https://archive.ics.uci.edu/ml/machine-learning-databases/secom/secom.data",
}


def load_dataset(name_or_url):
    """
    General entry point to load any supported CSSP dataset.

    Args:
        name_or_url (str): Either a key ('residential', 'secom') or a direct URL.

    Returns:
        A (np.ndarray): Correlation matrix (p x p).
        X_norm (np.ndarray): Normalized feature matrix (N x p).
    """
    # 1. Resolve URL
    url = DATASETS.get(name_or_url.lower(), name_or_url)

    print(f"  > Loading dataset: {name_or_url}")
    print(f"    Source: {url}")

    try:
        # 2. Download Data (Robust SSL handling)
        response = requests.get(url, verify=False)
        response.raise_for_status()
        content = io.BytesIO(response.content)

        # 3. Dispatch to specific loader based on known URLs
        if "Residential-Building" in url:
            X_raw = _parse_residential(content)
        elif "secom" in url:
            X_raw = _parse_secom(content)
        else:
            # Fallback: Try generic CSV loading
            print("    Unknown format. Attempting generic CSV load...")
            df = pd.read_csv(content)
            X_raw = df.select_dtypes(include=[np.number]).to_numpy()

        if X_raw is None:
            return None, None

        # 4. Standardize and Compute Correlation (Shared Logic)
        return _standardize_and_correlate(X_raw)

    except Exception as e:
        print(f"    CRITICAL ERROR loading data: {e}")
        return None, None


# --- SPECIFIC PARSERS ---


def _parse_residential(content):
    """Parser for UCI Residential Building (Excel, Headers, targets at end)."""
    try:
        # Read Excel (Header is on row 1, index 1)
        df = pd.read_excel(content, header=1)

        # Columns 4 to 107 are the features (V-1 to V-104)
        # Drop first 4 (ID/Dates) and last 2 (Targets)
        X_df = df.iloc[:, 4:107]

        # Force numeric
        X_raw = X_df.apply(pd.to_numeric, errors="coerce").to_numpy(dtype=np.float64)
        return np.nan_to_num(X_raw)

    except Exception as e:
        print(f"    Error parsing Residential Excel: {e}")
        return None


def _parse_secom(content):
    """Parser for SECOM (Space-separated, No Header, Constant Columns)."""
    try:
        # Read CSV with space delimiter
        df = pd.read_csv(content, sep="\s+", header=None)
        X_raw = df.to_numpy(dtype=np.float64)

        # Fill NaNs (SECOM has many)
        X_raw = np.nan_to_num(X_raw)

        # SECOM SPECIFIC: Drop constant columns (Variance = 0)
        # If we don't do this, the matrix is singular and solver crashes.
        std_devs = np.std(X_raw, axis=0)
        keep_idx = np.where(std_devs > 1e-9)[0]

        print(
            f"    [SECOM Cleaning] Dropped {X_raw.shape[1] - len(keep_idx)} constant columns."
        )
        return X_raw[:, keep_idx]

    except Exception as e:
        print(f"    Error parsing SECOM CSV: {e}")
        return None
    
    
def _standardize_and_correlate(X_raw):
    """
    Normalizes X (Z-score) and calculates A = (X^T X) / N.
    Used for ALL datasets to ensure consistent math.
    Note that it doesn't change anything, because N is a constant
    """
    N, p = X_raw.shape
    print(f"    Raw Data Shape: {N} rows x {p} features")

    # Z-score Normalization
    X_mean = np.mean(X_raw, axis=0)
    X_std = np.std(X_raw, axis=0)

    # Safety: Avoid division by zero
    X_std[X_std == 0] = 1.0

    X_norm = (X_raw - X_mean) / X_std

    # Correlation Matrix
    A = (X_norm.T @ X_norm) / N

    print(f"    Computed Correlation Matrix A: {A.shape}")
    return A, X_norm



<>:84: SyntaxWarning: "\s" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\s"? A raw string is also an option.
<>:84: SyntaxWarning: "\s" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\s"? A raw string is also an option.
/var/folders/7c/q9_nslk55p94jf29j5gtcsxc0000gn/T/ipykernel_9177/2036679825.py:84: SyntaxWarning: "\s" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\s"? A raw string is also an option.
  df = pd.read_csv(content, sep="\s+", header=None)


In [5]:
A, X_norm = load_dataset(DATASETS['secom'])

  > Loading dataset: https://archive.ics.uci.edu/ml/machine-learning-databases/secom/secom.data
    Source: https://archive.ics.uci.edu/ml/machine-learning-databases/secom/secom.data


/Users/nautilus/gridfw/.venv/lib/python3.14/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'archive.ics.uci.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


    [SECOM Cleaning] Dropped 112 constant columns.
    Raw Data Shape: 1567 rows x 478 features
    Computed Correlation Matrix A: (478, 478)


In [6]:
A

array([[ 1.        , -0.04778231,  0.0075897 , ...,  0.0052334 ,
         0.00391812,  0.0214887 ],
       [-0.04778231,  1.        , -0.03044774, ...,  0.03947934,
         0.04049988,  0.04446374],
       [ 0.0075897 , -0.03044774,  1.        , ..., -0.00709043,
        -0.00912185, -0.03043445],
       ...,
       [ 0.0052334 ,  0.03947934, -0.00709043, ...,  1.        ,
         0.97433188,  0.39150294],
       [ 0.00391812,  0.04049988, -0.00912185, ...,  0.97433188,
         1.        ,  0.3898976 ],
       [ 0.0214887 ,  0.04446374, -0.03043445, ...,  0.39150294,
         0.3898976 ,  1.        ]], shape=(478, 478))

In [7]:
X_norm

array([[ 0.13999828,  0.42920791,  0.03273532, ..., -1.86846064,
        -1.84076867, -1.0611589 ],
       [ 0.46402024, -0.10587396,  0.23685169, ...,  0.41258698,
         0.25102908,  1.15695081],
       [-0.35125598,  0.40723307,  0.02641324, ...,  3.62421124,
         3.31899911, -0.17840653],
       ...,
       [-0.12041844, -0.5678868 ,  0.12151173, ..., -0.89249002,
        -0.96918628, -0.59748491],
       [-0.53957345,  0.25606135, -0.01842658, ...,  0.91192079,
         0.77397852, -0.06511812],
       [-0.28974927, -0.18370602,  0.06960583, ..., -0.03000435,
        -0.27192036,  0.40672924]], shape=(1567, 478))